In [1]:
import os

In [2]:
def create_keyspace(name, replication_factor):
    return """
    CREATE KEYSPACE IF NOT EXISTS %s WITH replication = {
        'class': 'SimpleStrategy', 
        'replication_factor': '%s'
    };
    """ % (name, replication_factor)

In [3]:
create_keyspace('foo', 3)

"\n    CREATE KEYSPACE IF NOT EXISTS foo WITH replication = {\n        'class': 'SimpleStrategy', \n        'replication_factor': '3'\n    };\n    "

In [4]:
"""
CREATE KEYSPACE IF NOT EXISTS surface_data WITH replication = {
    'class': 'SimpleStrategy', 
    'replication_factor': '%s'
};
""" % ()

TypeError: not enough arguments for format string

In [ ]:
def read_file(path):
    with open(path, 'r') as content_file:
        return content_file.read()

files = os.listdir('cql')
files.sort()

for path in files:
    query = read_file(f'cql/{path}')
    print(query)
    print()

In [114]:
from confluent_kafka.admin import AdminClient, NewPartitions, NewTopic
from confluent_kafka import TopicPartition, Consumer

In [30]:
from confluent_kafka.admin import AdminClient, NewTopic

class create_admin_client:
    def __init__(self):
        
        # KAFKA INSTANCES
        ports = [11001, 11002]
        brokers = ','.join([f'localhost:{str(x)}' for x in ports])

        # ATTEMPT TO CONNECT TO THE CLUSTER
        self.instance = AdminClient({
            'bootstrap.servers': brokers,
        })
        
    # FETCH ALL EXISTING TOPICS
    def all_topics(self):
        container = {}
        
        for name, parts in self.instance.list_topics().topics.items():
            container[name] = len(parts.partitions)
        
        return container
    
    # CHECK IF TOPIC ALREADY EXISTS
    def topic_exists(self, target_topic):
        for topic in list(self.all_topics().keys()):
            if topic == target_topic:
                return True
        
        return False
    
    # ATTEMPT TO CREATE A NEW TOPIC
    def create_topic(self, name, num_partitions):

        # THROW ERROR IF TOPIC ALREADY EXISTS
        if self.topic_exists(name):
            raise Exception('ERROR: THIS TOPIC ALREADY EXISTS')

        # OTHERWISE, CREATE IT
        self.instance.create_topics(
            new_topics=[NewTopic(
                topic=name,
                num_partitions=num_partitions,
                replication_factor=1,
            )]
        )

In [83]:
kafka_admin = create_admin_client()

In [84]:
kafka_admin.all_topics()

{'eyylmao': 2, 'eyylmaoZ': 5, 'foobarz': 1, 'eyy': 2, 'foobar': 2}

In [ ]:
kafka_admin.create_topic(
    name='foobarz',
    num_partitions=1
)

In [36]:
import requests

In [79]:
def get_req(endpoint):
    response = requests.get(endpoint)
    
    print('STATUS:', response.status_code)
    return response.json()

In [80]:
def post_req(endpoint, body):
    response = requests.post(endpoint, json=body)
    
    print('STATUS:', response.status_code)
    return response.json()

In [81]:
get_req('http://localhost:3003/kafka')

STATUS: 200


{'eyylmao': 2, 'eyylmaoZ': 5, 'foobarz': 1, 'eyy': 2, 'foobar': 2}

In [82]:
post_req('http://localhost:3003/kafka/create', {
    'name': 'eyylmaoZ',
    'num_partitions': 5
})

STATUS: 500


'ERROR: TOPIC ALREADY EXISTS'